In [2]:
import pandas as pd

df_course = pd.read_csv('../assets/student.course.csv')
df_record = pd.read_csv('../assets/student.record.csv')
df_term = pd.read_table('../assets/term.table.txt', delimiter="\t").fillna('Unknown')

df_term.columns = ['TERM_ID', 'TERM_DESCRIPTION']

terms = ['Fall', 'Winter', 'Summer', 'Spring', 'Unknown']
df_term['TERM_NAME'] = df_term['TERM_DESCRIPTION'].apply(lambda x: list(filter(lambda y: y!='', [t if t in x else '' for t in terms]))[0])
df_term['TERM_YEAR'] = df_term['TERM_DESCRIPTION'].apply(lambda x: int(x[-4:]) if x[-4:].isnumeric() else None)

merge_course_df = pd.merge(
                    df_course,
                    df_term, 
                    how='left',
                    left_on='TERM',
                    right_on='TERM_ID')

terms_student = ['ADMIT_TERM', 'MAJOR1_TERM', 'MAJOR2_TERM', 'MAJOR3_TERM']
merge_record_df = df_record.copy()
for term in terms_student:
    
    merge_record_df = pd.merge(
                        merge_record_df,
                        df_term, 
                        how='left',
                        left_on=term,
                        right_on='TERM_ID')
    
    merge_record_df.rename(columns={'TERM_ID': term + '_' + 'TERM_ID', 
                            'TERM_DESCRIPTION': term + '_' + 'TERM_DESCRIPTION', 
                            'TERM_NAME': term + '_' + 'TERM_NAME', 
                            'TERM_YEAR': term + '_' + 'TERM_YEAR'}, inplace=True)    

# print unique df_course columns for reference 
print(df_course.columns)

# Print and exammine the first ten records of course dataframe
print(df_course.head(10))

# Based on our use case questions and the data contained in df_course data, 
# df_course data was excluded from the analysis for the following reasons:
# 1. Our use case is only concered with the volume of students by gender, graduation year, and major as compared to BLS workforce data
# 2. Non of our use case questions are concerned with the courses students take, only the outcome of their major

# print unique merge_record_df columns for reference 
print(merge_record_df.columns)

# record_df is used because it contains the volume of students by gender, graduation year, and major as compared to BLS workforce data


# Begin cleaning and manipulation of studet record data

merge_record_df.sort_values(by=['MAJOR3_TERM_TERM_YEAR'], ascending=[True])['MAJOR3_TERM_TERM_YEAR'].unique()


merge_record_df['ADMIT_TERM_EQUALS_MAJOR1_TERM'] = merge_record_df['ADMIT_TERM'] == merge_record_df['MAJOR1_TERM']
merge_record_df['MAJOR1_TERM_YEAR_MINUS_ADMIT_TERM_YEAR'] = merge_record_df['MAJOR1_TERM_TERM_YEAR'] - merge_record_df['ADMIT_TERM_TERM_YEAR']

# Take the count of all records before exclusions
print('number of student records before exclusions', len(merge_record_df))
print(merge_record_df[['ADMIT_TERM_EQUALS_MAJOR1_TERM', 'MAJOR1_TERM_YEAR_MINUS_ADMIT_TERM_YEAR']].describe())
# The mean of the difference between the Major1 graduation term year and the admit term year is 3.7 years,
# It is safe to assume that MAJOR1_TERM field is the graduation term of the student

# Measure only the Students who have a Major1 graduation term
merge_record_df = merge_record_df[merge_record_df.isnull()['MAJOR1_TERM'] == False]
print('number of student records excluding students without graduation term', len(merge_record_df))

# use box plot to identify outliers i.e. extreme values such as the -28 years minimum value as observed by describe() statistical function
merge_record_df['MAJOR1_TERM_YEAR_MINUS_ADMIT_TERM_YEAR'].unique()

# Exclude graduation terms with extreme values i.e. negative values
merge_record_df = merge_record_df[(merge_record_df.isnull()['MAJOR1_TERM_YEAR_MINUS_ADMIT_TERM_YEAR'] == False)\
                                  & (merge_record_df['MAJOR1_TERM_YEAR_MINUS_ADMIT_TERM_YEAR'] >= 0)]

print('number of student records excluding extreme values', len(merge_record_df))
merge_record_df[['ADMIT_TERM_TERM_YEAR','MAJOR1_TERM_TERM_YEAR','MAJOR2_TERM_TERM_YEAR','MAJOR3_TERM_TERM_YEAR']]

# There are 4 school terms:
# - ADMIT_TERM: The term the student was admitted to the school
# - MAJOR1_TERM: The term the student declared their first major
# - MAJOR2_TERM: The term the student declared their first major
# - MAJOR3_TERM: The term the student declared their first major







Index(['ANONID', 'SUBJECT', 'CATALOG_NBR', 'GRD_PTS_PER_UNIT', 'GPAO', 'DIV',
       'ANON_INSTR_ID', 'TERM'],
      dtype='object')
   ANONID SUBJECT  CATALOG_NBR  GRD_PTS_PER_UNIT      GPAO DIV  ANON_INSTR_ID  \
0      26     ACC          272               2.0  3.343636   P           2920   
1     114     ACC          272               2.0  2.817857   P           2920   
2     121     ACC          272               4.0  4.000000   P            201   
3     125     ACC          272               1.3  3.639063   P           3360   
4     180     ACC          272               3.0  3.846154   P           2920   
5     207     ACC          272               2.0  2.715686   P           2920   
6     224     ACC          272               3.0  3.591379   P           3604   
7     249     ACC          272               3.0  3.334483   P           1914   
8     356     ACC          272               4.0  3.655000   P           2640   
9     400     ACC          272               3.3  3.65000

,ADMIT_TERM_TERM_YEAR,MAJOR1_TERM_TERM_YEAR,MAJOR2_TERM_TERM_YEAR,MAJOR3_TERM_TERM_YEAR
1,2008.0,2012.0,NaN,NaN
2,1998.0,2002.0,NaN,NaN
3,2007.0,2012.0,NaN,NaN
7,2008.0,2012.0,NaN,NaN
8,1998.0,2001.0,2001.0,NaN
...,...,...,...,...
138880,1999.0,2003.0,NaN,NaN
138881,2000.0,2001.0,2001.0,2001.0
138882,2011.0,2015.0,NaN,NaN
138885,2005.0,2009.0,NaN,NaN


In [3]:

term = 'MAJOR1_TERM_TERM_YEAR' # ADMIT_TERM_TERM_YEAR'
merge_record_df['MAJOR1_DESCR'] = merge_record_df['MAJOR1_DESCR'].fillna('Undeclared')
df_majors = pd.read_excel('../assets/majors.xlsx', sheet_name='majors', header=0)

df_record_occupation = pd.merge(merge_record_df,
                            df_majors, 
                            how='left',
                            left_on='MAJOR1_DESCR',
                            right_on='MAJOR')

df_record_occupation['number_of_students_all'] = 1
df_record_occupation['number_of_students_men'] = df_record_occupation['SEX'].apply(lambda sex: 1 if sex == 'M' else 0) 
df_record_occupation['number_of_students_women'] = df_record_occupation['SEX'].apply(lambda sex: 1 if sex == 'F' else 0)
df_record_occupation['number_of_students_unknown'] = df_record_occupation['SEX'].apply(lambda sex: 1 if sex not in ['M', 'F'] else 0)
df_record_occupation['MAJOR1_TERM_TERM_YEAR'] = df_record_occupation['MAJOR1_TERM_TERM_YEAR'].apply(lambda x: str(x)[0:4])

df_record_occupation_grouped = df_record_occupation.groupby(['OCCUPATION',term]).sum().reset_index()

df_occupation_level_mapping = pd.read_excel('../assets/bls_cpsaat39_2011_to_2015.xlsx', sheet_name='level_mapping_l0', header=0)
df_occupation_level_mapping_distinct = df_occupation_level_mapping[['l4', 'l3', 'l2', 'l1']].drop_duplicates().reset_index()
df_occupation_level_mapping_distinct = df_occupation_level_mapping_distinct[['l4', 'l3', 'l2', 'l1']]

df_record_occupation_level_grouped = pd.merge(df_record_occupation_grouped,
                                                df_occupation_level_mapping_distinct, 
                                                how='left',
                                                left_on='OCCUPATION',
                                                right_on='l1')

df_record_occupation_level_grouped = df_record_occupation_level_grouped[[term, 'OCCUPATION', 'number_of_students_all', 
                                                                        'number_of_students_men', 'number_of_students_women', 
                                                                        'number_of_students_unknown', 'l4', 'l3', 'l2', 'l1']].\
                                                                        sort_values(by=['OCCUPATION', term])

df_record_occupation_level_grouped_by_year = df_record_occupation_level_grouped.groupby([term, 'l1']).sum().reset_index()
df_record_occupation_level_grouped_by_year=df_record_occupation_level_grouped_by_year[[term, 'l4', 'l3', 'l2', 'l1', 
                                                                                      'number_of_students_all', 'number_of_students_men', 
                                                                                      'number_of_students_women', 'number_of_students_unknown']]
df_record_occupation_level_grouped_by_year_filtered = df_record_occupation_level_grouped_by_year[df_record_occupation_level_grouped_by_year['MAJOR1_TERM_TERM_YEAR'].isin(['2011', '2012', '2013', '2014', '2015'])] 
# df_record_occupation_level_grouped_by_year_filtered


In [82]:




tabs = ['2015', '2014', '2013', '2012', '2011']

df_bls_all = pd.DataFrame()
for tab in tabs:
    df_bls_next = pd.read_excel('../assets/bls_cpsaat39_2011_to_2015.xlsx', 
                                sheet_name=str(tab), 
                                header=0, 
                                ).fillna('Unknown')
    df_bls_next['year'] = str(tab)
    df_bls_all = pd.concat([df_bls_all, df_bls_next])


df_level = pd.merge(df_bls_all,
                    df_occupation_level_mapping, 
                    how='left',
                    left_on='occupation',
                    right_on='l0')

df_level = df_level[df_level['l0'].notnull()]


df_level['year'] = df_level['year'].apply(lambda x: str(x))

df_level = df_level.groupby(['year', 'l1'], sort=True).agg({
    'number_of_workers_all': 'sum',
    'median_weekly_earnings_all': 'mean',
    'number_of_workers_men': 'sum',
    'median_weekly_earnings_men': 'mean',
    'number_of_workers_women': 'sum',
    'median_weekly_earnings_women': 'mean',
    'occupation_x': 'first',
    'occupation_y': 'first',
    'l4': 'first',
    'l3': 'first',
    'l2': 'first',
    'l0': 'first'
}).reset_index()

df_merge_bls = pd.merge(df_level,
                    df_record_occupation_level_grouped_by_year_filtered, 
                    how='left',
                    left_on=['l1', 'year'],
                    right_on=['l1', 'MAJOR1_TERM_TERM_YEAR'])[[
                                'year', 'occupation_x', 'l0','l1','l2_x','l3_x','l4_x',                                    
                                'number_of_workers_all', 'median_weekly_earnings_all',
                                'number_of_workers_men', 'median_weekly_earnings_men',
                                'number_of_workers_women', 'median_weekly_earnings_women',
                                'number_of_students_all', 'number_of_students_men',
                                'number_of_students_women', 'number_of_students_unknown'
                    ]]

new_column_names = {
    'year': 'year',
    'occupation_x': 'occupation',
    'l0': 'l0',
    'l1': 'l1',
    'l2_x': 'l2',
    'l3_x': 'l3',
    'l4_x': 'l4',
    'number_of_workers_all': 'number_of_workers_all_sum',
    'median_weekly_earnings_all': 'median_weekly_earnings_all_mean',
    'number_of_workers_men': 'number_of_workers_men_sum',
    'median_weekly_earnings_men': 'median_weekly_earnings_men_mean',
    'number_of_workers_women': 'number_of_workers_women_sum',
    'median_weekly_earnings_women': 'median_weekly_earnings_women_mean',
    'number_of_students_all': 'number_of_students_all_sum',
    'number_of_students_men': 'number_of_students_men_sum',
    'number_of_students_women': 'number_of_students_women_sum',
    'number_of_students_unknown': 'number_of_students_unknown_sum'
}


df_merge_bls.rename(columns=new_column_names, inplace=True)

df_merge_bls


df_level_list = []
selected_levels = ['l1','l2','l3']
for level in selected_levels:
    
    df_merge_bls_level = df_merge_bls[[level, 'year',                              
                        'number_of_students_all_sum',
                        'number_of_students_men_sum', 
                        'number_of_students_women_sum', 
                        'number_of_students_unknown_sum',
                        'number_of_workers_all_sum', 
                        'median_weekly_earnings_all_mean', 
                        'number_of_workers_men_sum', 
                        'median_weekly_earnings_men_mean', 
                        'number_of_workers_women_sum', 
                        'median_weekly_earnings_women_mean']
                        ]

    df_merge_bls_grouped = df_merge_bls_level.groupby(['year', level],  sort=True).sum().reset_index()
    df_level_list.append(df_merge_bls_grouped)
    df_merge_bls_grouped.to_csv('../scratch/df_merge_bls_'+ level + '.csv', index=False)
    
# df_level_list[0]


In [80]:

def get_short_names(level, metric) -> dict:
    
    short_names_dict = { 
        'l1': {
            'default':{
                'Architecture and engineering occupations' : 'Arch & Eng',
                'Arts, design, entertainment, sports, and media occupations' : 'Ent',
                'Building and grounds cleaning and maintenance occupations' : 'Maintenance',
                'Business and financial operations occupations' : 'Financial',
                'Community and social service occupations' : 'Social Services',
                'Computer and mathematical occupations' : 'Comp & Math',
                'Construction and extraction occupations' : 'Construction',
                'Education, training, and library occupations' : 'Education',
                'Farming, fishing, and forestry occupations' : 'Natural Resources',
                'Food preparation and serving related occupations' : 'Food',
                'Healthcare practitioners and technical occupations' : 'Healthcare Practitioners',
                'Healthcare support occupations' : 'Healthcare',
                'Installation, maintenance, and repair occupations' : 'Facilities',
                'Legal occupations' : 'Legal',
                'Life, physical, and social science occupations' : 'Science',
                'Management occupations' : 'Management',
                'Office and administrative support occupations' : 'Office Admin',
                'Personal care and service occupations': 'Personal Care', 
                'Production occupations' : 'Production',
                'Protective service occupations': 'Security', 
                'Sales and related occupations' : 'Sales',
                'Transportation and material moving occupations' : 'Logistics'
            },            
            'number_of_workers':{
                'Architecture and engineering occupations' : 'Arch & Eng',
                'Arts, design, entertainment, sports, and media occupations' : 'Ent',
                'Building and grounds cleaning and maintenance occupations' : 'Maintenance',
                'Business and financial operations occupations' : 'Financial',
                'Community and social service occupations' : 'Social Srvcs',
                'Computer and mathematical occupations' : 'Comp & Math',
                'Construction and extraction occupations' : 'Construction',
                'Education, training, and library occupations' : 'Education',
                'Farming, fishing, and forestry occupations' : 'Ntrl Rsrcs',
                'Food preparation and serving related occupations' : 'Food',
                'Healthcare practitioners and technical occupations' : 'Healthcare Practitioners',
                'Healthcare support occupations' : 'Healthcare',
                'Installation, maintenance, and repair occupations' : 'Facilities',
                'Legal occupations' : 'Legal',
                'Life, physical, and social science occupations' : 'Sci',
                'Management occupations' : 'Management',
                'Office and administrative support occupations' : 'Office Admin',
                'Personal care and service occupations': 'Personal Care', 
                'Production occupations' : 'Production',
                'Protective service occupations': 'Security', 
                'Sales and related occupations' : 'Sales',
                'Transportation and material moving occupations' : 'Logistics'
            },
            'number_of_students':{
                'Architecture and engineering occupations' : 'Arch & Eng',
                'Arts, design, entertainment, sports, and media occupations' : 'Ent',
                'Building and grounds cleaning and maintenance occupations' : 'Maintenance',
                'Business and financial operations occupations' : 'Fin',
                'Community and social service occupations' : 'Social Services',
                'Computer and mathematical occupations' : 'Computer',
                'Construction and extraction occupations' : 'Construction',
                'Education, training, and library occupations' : 'Education',
                'Farming, fishing, and forestry occupations' : 'Natural Resources',
                'Food preparation and serving related occupations' : 'Food',
                'Healthcare practitioners and technical occupations' : 'Med Pract',
                'Healthcare support occupations' : 'Healthcare',
                'Installation, maintenance, and repair occupations' : 'Facilities',
                'Legal occupations' : 'Legal',
                'Life, physical, and social science occupations' : 'Science',
                'Management occupations' : 'Mgmt',
                'Office and administrative support occupations' : 'Office Admin',
                'Personal care and service occupations': 'Personal Care', 
                'Production occupations' : 'Production',
                'Protective service occupations': 'Security', 
                'Sales and related occupations' : 'Sales',
                'Transportation and material moving occupations' : 'Logistics'
            },            
        },
        'l2': {
            'default':{
                'Building and grounds cleaning and maintenance occupations' : 'Maint',
                'Construction and extraction occupations' : 'Construction',
                'Farming, fishing, and forestry occupations' : 'Natural Resources',
                'Food preparation and serving related occupations' : 'Food',
                'Healthcare support occupations' : 'Health',
                'Installation, maintenance, and repair occupations' : 'Facilities',
                'Management, business, and financial operations occupations' : 'Business',
                'Office and administrative support occupations' : 'Office Admin',
                'Personal care and service occupations': 'Personal Care', 
                'Production occupations' : 'Production',
                'Professional and related occupations' : 'Professional',
                'Protective service occupations' : 'Security',
                'Sales and related occupations' : 'Sales',
                'Transportation and material moving occupations' : 'Logistics'
            },            
            'number_of_workers':{
                'Building and grounds cleaning and maintenance occupations' : 'Maint',
                'Construction and extraction occupations' : 'Construction',
                'Farming, fishing, and forestry occupations' : 'Natural',
                'Food preparation and serving related occupations' : 'Food',
                'Healthcare support occupations' : 'Health',
                'Installation, maintenance, and repair occupations' : 'Facilities',
                'Management, business, and financial operations occupations' : 'Business',
                'Office and administrative support occupations' : 'Office Admin',
                'Personal care and service occupations': 'Personal Care', 
                'Production occupations' : 'Production',
                'Professional and related occupations' : 'Professional',
                'Protective service occupations' : 'Security',
                'Sales and related occupations' : 'Sales',
                'Transportation and material moving occupations' : 'Logistics'
            },
            'number_of_students':{
                'Building and grounds cleaning and maintenance occupations' : 'Maint',
                'Construction and extraction occupations' : 'Construction',
                'Farming, fishing, and forestry occupations' : 'Natural Resources',
                'Food preparation and serving related occupations' : 'Food',
                'Healthcare support occupations' : 'Health',
                'Installation, maintenance, and repair occupations' : 'Facilities',
                'Management, business, and financial operations occupations' : 'Bus',
                'Office and administrative support occupations' : 'Office',
                'Personal care and service occupations': 'Personal Care', 
                'Production occupations' : 'Production',
                'Professional and related occupations' : 'Professional',
                'Protective service occupations' : 'Security',
                'Sales and related occupations' : 'Sales',
                'Transportation and material moving occupations' : 'Logistics'
            },

        },
    }

    return short_names_dict[level][metric] if metric in short_names_dict[level].keys() else short_names_dict[level]['default']
    
from bokeh.palettes import Cividis256


 

def get_palette(number_of_groups) -> dict:

    Cividis = {
        'default': Cividis256[0:3] + Cividis256[64:67] + Cividis256[128:131]+ Cividis256[192: 195] + Cividis256[252: 255],
        14: Cividis256[0:3] + Cividis256[64:67] + Cividis256[128:131]+ Cividis256[192: 195] + Cividis256[252: 255],
        4: [Cividis256[0]] + [Cividis256[64]] + [Cividis256[192]] + [Cividis256[252]],
        2: Cividis256[0]  + Cividis256[255]
    }

    return Cividis[number_of_groups] if number_of_groups in Cividis.keys() else Cividis['default']


def get_format_parameters(metric, number_of_groups) -> dict:
    
    palette = get_palette(number_of_groups=number_of_groups)
    print(number_of_groups,' - ', palette)
    label_offset_dict = { 
        'default':{
                'block':{'line_width': 1, 
                         'line_color': 'white', 
                         'fill_alpha': 0.80, 
                         'palette': palette},
                'l2' : {'x_offset': 2, 
                        'y_offset': -35,
                        'text_font_size': "30pt",
                        'text_baseline': "top",
                        'text_color': 'black'
                        },
                'l1' : {'x_offset': 2, 
                        'y_offset': 35,
                        'text_font_size': "18pt",
                        'text_baseline': "top",
                        'text_color': 'black'
                        },
        },            

        'number_of_workers':{
            'block':{'line_width': 10, 
                        'line_color': 'white', 
                        'fill_alpha': 0.80, 
                        'palette': palette},
            'l2' : {'x_offset': 8, 
                    'y_offset': -15,
                    'text_font_size': "24pt",
                    'text_baseline': "top",
                    'text_color': 'black'
                    },
            'l1' : {'x_offset': 8, 
                    'y_offset': 4,
                    'text_font_size': "18pt",
                    'text_baseline': "top",
                    'text_color': 'white'
                    },
        },
        
        'number_of_students':{
                'block':{'line_width': 10, 
                         'line_color': 'white', 
                         'fill_alpha': 0.80, 
                         'palette': palette},
                'l2' : {'x_offset': 10, 
                        'y_offset': -35,
                        'text_font_size': "30pt",
                        'text_baseline': "top",
                        'text_color': 'black'
                        },
                'l1' : {'x_offset': 10, 
                        'y_offset': 4,
                        'text_font_size': "18pt",
                        'text_baseline': "top",
                        'text_color': 'white'
                        },
        },
    }
        
    return label_offset_dict[metric] if metric in label_offset_dict.keys() else label_offset_dict[metric]['default']






In [81]:



import pandas as pd
from squarify import normalize_sizes, squarify

from bokeh.plotting import figure, show

from bokeh.sampledata.sample_superstore import data
from bokeh.transform import factor_cmap
from bokeh.io import output_notebook
from jupyter_bokeh.widgets import BokehModel # widgets third party extension will be required
from bokeh.transform import linear_cmap
from bokeh.models import CrosshairTool, Span, NumeralTickFormatter, HoverTool, CustomJS, Slider, ColumnDataSource


df_occupation_level_mapping = pd.read_excel('../assets/bls_cpsaat39_2011_to_2015.xlsx', sheet_name='level_mapping_l0', header=0)
df_occupation_level_mapping_distinct = df_occupation_level_mapping[['l4', 'l3', 'l2', 'l1']].drop_duplicates().reset_index()
df_occupation_level_mapping_distinct = df_occupation_level_mapping_distinct[['l4', 'l3', 'l2', 'l1']]
df_occupation_level_mapping_distinct

# metric = 'number_of_students_women_sum'
metric = 'number_of_workers_all_sum'
base_metric = metric.replace('_sum', '').replace('_women', '').replace('_men', '').replace('_all', '')

l1_grouping = df_level_list[0]
l2_grouping = df_level_list[1]

l1_grouping = pd.merge(l1_grouping,
                        df_occupation_level_mapping_distinct[['l1', 'l2']], 
                        how='left',
                        left_on=['l1'],
                        right_on=['l1'])


l1_grouping = l1_grouping[l1_grouping['year'] == '2015']
l1_grouping = l1_grouping[['l1','l2', metric]]
l2_grouping = l2_grouping[l2_grouping['year'] == '2015']
l2_grouping = l2_grouping[['l2', metric]]

short_name_l2 = get_short_names('l2', base_metric)
l1_grouping['l2'] = l1_grouping['l2'].apply(lambda x: short_name_l2[x] if x in short_name_l2.keys() else x)
l2_grouping['l2'] = l2_grouping['l2'].apply(lambda x: short_name_l2[x] if x in short_name_l2.keys() else x)

short_name_l1 = get_short_names('l1', base_metric)
l1_grouping['l1'] = l1_grouping['l1'].apply(lambda x: short_name_l1[x] if x in short_name_l1.keys() else x)


total = l2_grouping[metric].sum() # total seems low, validate later on

l1_grouping['l1'] = l1_grouping.apply(lambda x: x['l1'] + ' | ' + str(int(round(x[metric]/total*100, 0))) + '%', axis=1)
l2_grouping['l2'] = l2_grouping.apply(lambda x: x['l2'] + ' | ' + str(int(round(x[metric]/total*100, 0))) + '%', axis=1)
l2_lookup = {k: k + " | " + v for k, v in (x.split(" | ") for x in l2_grouping['l2'].to_list())}
l1_grouping['l2'] = l1_grouping.apply(lambda x: l2_lookup[x['l2']], axis=1)

# l1_grouping



# keep only records with non-zero values, otherwise treemap will throw a divide by zero error
l1_grouping = l1_grouping[l1_grouping[metric] > 0]
l2_grouping = l2_grouping[l2_grouping[metric] > 0]

def treemap(df, col, x, y, dx, dy, *, N=100):
    sub_df = df.nlargest(N, col)
    normed = normalize_sizes(sub_df[col], dx, dy)
    blocks = squarify(normed, x, y, dx, dy)
    blocks_df = pd.DataFrame.from_dict(blocks).set_index(sub_df.index)
    return sub_df.join(blocks_df, how='left').reset_index()


# for sex in ['women','men','all']:
#     if sex in metric:
#         l2_grouping['l2'] = l2_grouping['l2'].apply(lambda x: x + ' ' + sex.title()) 
#         break

# for sex in ['women','men','all']:
#     if sex in metric:
#         l1_grouping['l2'] = l1_grouping['l2'].apply(lambda x: x + ' ' + sex.title()) 
#         break


l2s = tuple(l2_grouping['l2'].unique().tolist())


x, y, w, h = 0, 0, 2000, 1125
blocks_by_L2 = treemap(l2_grouping, metric, x, y, w, h)

blocks_by_L2.drop(columns=['index'], inplace=True)

blocks_by_L2 = blocks_by_L2[['l2', metric, 'x', 'y', 'dx', 'dy']] 

dfs = []
for index, (l2, number_of_workers_all, x, y, dx, dy) in blocks_by_L2.iterrows():
    df = l1_grouping[l1_grouping.l2==l2]    
    dfs.append(treemap(df, metric, x, y, dx, dy, N=100))

blocks = pd.concat(dfs)


p = figure(width=w, height=h, tooltips="@l1", toolbar_location=None,
           x_axis_location=None, y_axis_location=None)
p.x_range.range_padding = p.y_range.range_padding = 0
p.grid.grid_line_color = None


# Get treemap format parameters
param_set = get_format_parameters(metric=base_metric, number_of_groups=len(l2_grouping))
block_params = param_set['block']
l2_params = param_set['l2']
l1_params = param_set['l1']

p.block('x', 'y', 'dx', 'dy', source=blocks, line_width=block_params['line_width'], line_color=block_params['line_color'],
        fill_alpha=block_params['fill_alpha'], fill_color=factor_cmap("l2", block_params['palette'][::-1] if 'women' in metric else block_params['palette'], l2s)) #legend_field="l2",

p.text('x', 'y',  x_offset=l2_params['x_offset'], y_offset=l2_params['y_offset'], text="l2", source=blocks_by_L2,
       text_font_size=l2_params['text_font_size'], text_color=l2_params['text_color'])

blocks["ytop"] = blocks.y + blocks.dy

p.text('x', 'ytop', x_offset=l1_params['x_offset'], y_offset=l1_params['y_offset'], text="l1", source=blocks,
       text_font_size=l1_params['text_font_size'], text_baseline=l1_params['text_baseline'],
       text_color=l1_params['text_color'])

# p.legend.title = "L2 Grouping"
# p.legend.label_text_font_size = "10pt"
# p.legend.location = "top_right"
# p.legend.orientation = "vertical"


# Generate color band
# y = l2_grouping[metric]
# cmap = linear_cmap(field_name='y', palette=get_palette(len(l2_grouping)), low=min(y), high=max(y))
# r = p.scatter(l2_grouping['l2'], y, color=cmap, size=25, legend_label='Occupation')
# color_bar = r.construct_color_bar(width=10) #, formatter=NumeralTickFormatter(format="0,0")
# p.add_layout(color_bar, 'right')    

output_notebook()
handle = display(BokehModel(p))
blocks

blocks_by_L2





14  -  ('#00204C', '#00204E', '#002150', '#414D6B', '#424E6B', '#434E6B', '#7C7B78', '#7D7C78', '#7E7D78', '#BDAF6E', '#BEB06E', '#BFB16D', '#FFE642', '#FFE743', '#FFE844')


Loading BokehJS ...

BokehModel(render_bundle={'docs_json': {'42466ebc-dc9a-4d5c-bcf6-d95a40500251': {'version': '3.4.3', 'title': …

,l2,number_of_workers_all_sum,x,y,dx,dy
0,Professional | 24%,26566,0.000000,0.000000,824.839801,664.327154
1,Business | 17%,18422,0.000000,664.327154,824.839801,460.672846
2,Office Admin | 13%,13832,824.839801,0.000000,431.946316,660.511906
3,Sales | 9%,9727,824.839801,660.511906,431.946316,464.488094
4,Production | 7%,7472,1256.786117,0.000000,384.923694,400.394094
5,Logistics | 6%,6955,1641.709811,0.000000,358.290189,400.394094
6,Construction | 5%,5721,1256.786117,400.394094,285.341299,413.554508
7,Facilities | 4%,4303,1256.786117,813.948602,285.341299,311.051398
8,Food | 4%,4124,1542.127416,400.394094,244.372530,348.090136
9,Maint | 3%,3603,1786.499945,400.394094,213.500055,348.090136
